# Scratch Book

**Author:** Andrew Loeppky (Lots of code stolen from Jamie Byer)

**Project:** Land-surface-atmosphere coupling - CMIP6 intercomparison 

experiment space for writing, testing code related to Betts CMIP6 intercomparison project

**Workflow:** 

1) get raw xarrays using Jamie's model fetching code. Screen models which do not contain required fields for doing the calculations

2) convert to metpy CF standards using `metpy.parse_cf(<raw_xarray>)`. Generate the necessary fields to make the figures we want. 

3) pare fields down to the most naive data type allowable (numpy arrays?) and plot with matplotlib (not some weird wrapper for matplotlib, too buggy).

## Part I: Get a CMIP 6 Dataset and Select Domain

In [1]:
import xarray as xr
import pooch
import pandas as pd
import fsspec
from pathlib import Path
import time
import numpy as np
import json
import cftime
import matplotlib.pyplot as plt
import netCDF4 as nc


# Handy metpy tutorial working with xarray:
# https://unidata.github.io/MetPy/latest/tutorials/xarray_tutorial.html#sphx-glr-tutorials-xarray-tutorial-py
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.units import units
from metpy.plots import SkewT

In [2]:
# Attributes of the model we want to analyze (put in csv later)
#source_id = 'CESM2-SE'
source_id = 'GFDL-ESM4'
experiment_id = 'piControl'
#table_id = 'Amon'
table_id = '3hr'

# Domain we wish to study
lats = (15, 20) # lat min, lat max
lons = (25, 29) # lon min, lon max
years = (100, 105) # start year, end year (note, no leap days)
#ceil = 500 # top of domain, hPa


print(f"""Fetching domain:
          {source_id = }
          {experiment_id = }
          {table_id = }
          {lats = }
          {lons = }
          {years = }
          dataset name: my_ds (xarray Dataset)""")

Fetching domain:
          source_id = 'GFDL-ESM4'
          experiment_id = 'piControl'
          table_id = '3hr'
          lats = (15, 20)
          lons = (25, 29)
          years = (100, 105)
          dataset name: my_ds (xarray Dataset)


In [3]:
# list of fields required for input calculations
required_fields = ("ps",  # surface pressure
                      "cl",  # cloud fraction
                      "ta",  # air temperature
                      "ts",  # surface temperature
                      "hus", # specific humidity
                      "hfls", # Surface Upward Latent Heat Flux
                      "hfss", # Surface Upward Sensible Heat Flux
                      "rlds",  # surface downwelling longwave
                      "rlus",  # surface upwelling longwave
                      "rsds", # downwelling short wave
                      "rsus", # upwelling short wave
                      "hurs",  # near surface RH
                      "pr", # precipitation, all phases
                      "evspsbl", # evaporation, sublimation, transpiration
                      "wap",  # omega (subsidence rate in pressure coords)
                   )

required_fields = ['tas', 'mrsos', 'mrro', 'tslsi', 'huss'] # temporary hack, but this will work for fig 11
# i need to know which models we intend to parse for this project, they do not all have the same fields

In [4]:
# get esm datastore
odie = pooch.create(
    path="./.cache",
    base_url="https://storage.googleapis.com/cmip6/",
    registry={
        "pangeo-cmip6.csv": None
    },
)
file_path = odie.fetch("pangeo-cmip6.csv")
df_in = pd.read_csv(file_path)

In [5]:
# extract the names of all fields in our selected model run
available_fields = list(df_in[df_in.source_id == source_id][df_in.experiment_id == experiment_id][df_in.table_id == table_id].variable_id)

/tmp/ipykernel_6349/3986708080.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  available_fields = list(df_in[df_in.source_id == source_id][df_in.experiment_id == experiment_id][df_in.table_id == table_id].variable_id)


In [6]:
available_fields

['tas', 'mrsos', 'mrro', 'tslsi', 'huss']

In [7]:
# check that our run has all required fields, list problem variables
fields_of_interest = []
missing_fields = []
for rq in required_fields:
    if rq not in available_fields:
        missing_fields.append(rq)
    else:
        fields_of_interest.append(rq)

if missing_fields != []:
    print(f"""WARNING: data from model run:

                {source_id}, 
                {table_id}, 
                {experiment_id} 

         missing required field(s): {missing_fields}""")

In [8]:
def fetch_var_exact(the_dict,df_og):
    the_keys = list(the_dict.keys())
    #print(the_keys)
    key0 = the_keys[0]
    #print(key0)
    #print(the_dict[key0])
    hit0 = df_og[key0] == the_dict[key0]
    if len(the_keys) > 1:
        hitnew = hit0
        for key in the_keys[1:]:
            hit = df_og[key] == the_dict[key]
            hitnew = np.logical_and(hitnew,hit)
            #print("total hits: ",np.sum(hitnew))
    else:
        hitnew = hit0
    df_result = df_og[hitnew]
    return df_result

In [9]:
def get_field(variable_id, 
              df,
              source_id=source_id,
              experiment_id=experiment_id,
              table_id=table_id):
    """
    extracts a single variable field from the model
    """

    var_dict = dict(source_id = source_id, variable_id = variable_id,
                    experiment_id = experiment_id, table_id = table_id)
    
    local_var = fetch_var_exact(var_dict, df)
    zstore_url = local_var['zstore'].array[0]
    the_mapper=fsspec.get_mapper(zstore_url)
    local_var = xr.open_zarr(the_mapper, consolidated=True)
    return local_var

In [10]:
def trim_field(df, lat, lon, years):
    """
    cuts out a specified domain from an xarrray field
    
    lat = (minlat, maxlat)
    lon = (minlon, maxlon)
    """
    new_field = df.sel(lat=slice(lat[0],lat[1]), lon=slice(lon[0],lon[1]))
    new_field = new_field.isel(time=(new_field.time.dt.year > years[0]))
    new_field = new_field.isel(time=(new_field.time.dt.year < years[1]))
    return new_field

In [11]:
# grab all fields of interest and combine
my_fields = [get_field(field, df_in) for field in fields_of_interest]
small_fields = [trim_field(field, lats, lons, years) for field in my_fields]
my_ds = xr.combine_by_coords(small_fields, compat="broadcast_equals", combine_attrs="drop_conflicts")
print("Successfully acquired domain")

Successfully acquired domain


In [60]:
from cftime import date2num
date2num(my_ds.time, "minutes since 0000-01-01 00:00:00", calendar="noleap", has_year_zero=True)


array([53085600, 53085690, 53085780, ..., 55187730, 55187820, 55187910])

In [61]:
# save as netcdf as per these recommendations:
# https://xarray.pydata.org/en/stable/user-guide/dask.html#chunking-and-performance
# this sucks. netcdf cant handle cftime, but it is the only format xarray seems to tolerate. ugh
my_ds["time"] = date2num(my_ds.time, "minutes since 0000-01-01 00:00:00", calendar="noleap", has_year_zero=True)
#my_ds["datetimeindex"] = my_ds.indexes['time'].to_datetimeindex()
my_ds.to_netcdf(f"./data/{source_id}-{experiment_id}")

TypeError: float() argument must be a string or a number, not 'cftime._cftime.DatetimeNoLeap'

## Part II: Convert to MetPy Standards and Copy Betts Fig 11

moved to `make_fields.ipynb`

In [ ]:
#specific_humidity = hourly_data.huss[np.isnan(hourly_data.huss.values) == False]
#surface_temp = hourly_data.tas[np.isnan(hourly_data.tas.values) == False]
#td = mpcalc.dewpoint_from_specific_humidity(ps, surface_temp, specific_humidity)

In [ ]:
#plcl, tlcl = mpcalc.lcl(ps, surface_temp, td)

In [ ]:
# add variables to generate fig 11
#dparsed["td"] = mpcalc.dewpoint_from_specific_humidity(ps, dparsed.tas.metpy.convert_units("kelvin"), dparsed.huss / 1000)

In [ ]:
# take spatial average over domain, group by hour and average each hour over time domain
#spatial_average = dparsed.mean(dim=("lat", "lon"))
# need to add a step here to select only warm months with PBL development
#hourly_data = spatial_average.groupby(dparsed.time.dt.hour).mean(dim="time")

In [ ]:
#plcl, tlcl = mpcalc.lcl(ps, spatial_average.tas * units.kelvin, spatial_average.td)

In [ ]:
#plot_me = np.array(plcl)[np.isnan(np.array(plcl)) == False]

In [ ]:
#plt.plot(plot_me)
#my_array = np.array([1, 2, np.nan])
#my_array[np.isnan(my_array) == False]

In [ ]:
#fig, ax = plt.subplots()
#ax.plot(hourly_data.hour[np.isnan(hourly_data.huss.values) == False], plot_this,)

In [ ]:
#warm_months = np.array([5,6,7,8,9])
#dparsed.isel(time=(dparsed.time.dt.month == warm_months.any()))

In [32]:
help(cftime.DatetimeNoLeap)


Help on class DatetimeNoLeap in module cftime._cftime:

class DatetimeNoLeap(datetime)
 |  DatetimeNoLeap(*args, **kwargs)
 |  
 |  Phony datetime object which mimics the python datetime object,
 |  but uses the "noleap" ("365_day") calendar.
 |  
 |  Method resolution order:
 |      DatetimeNoLeap
 |      datetime
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __pyx_vtable__ = <capsule object NULL>
 |  
 |  ----------------------------------------------------------------------
 |  Methods inh